In [55]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

import tensorflow as tf
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt

from sklearn import svm
import skimage as sk
import numpy as np
import argparse
import imutils
import pandas as panda
import cv2
import os
import random
from sklearn.model_selection import train_test_split

In [4]:
target_img_size = (32, 32)
random_seed = 42  
random.seed(random_seed)
np.random.seed(random_seed)

In [243]:
def cleanImage(image, label = ""):
    # threshold = sk.filters.threshold_otsu(image)

    threshold = 0.5
    image = sk.transform.resize(image, target_img_size, mode='reflect', anti_aliasing=True)

    # print(np.histogram(image.flatten()))
    if np.max(image) <= 1 and threshold > 1:
        threshold = threshold / 255
    
    
    image[image > threshold] = 1
    image[image <= threshold] = 0

    # image = sk.morphology.skeletonize(image)

    # if (random.random() > 0.9 and label != "") :
    # sk.io.imshow(image)
    # sk.io.show()

    # if image.dtype != np.uint8:
    #     image = image.astype(np.uint8)

    # image = cv2.Canny(image, 0.5 * threshold, threshold)
        

    return image

In [244]:
def extract_hog_features(img):
    
    if img.dtype != np.uint8:
        img = np.uint8(img)

    win_size = (16, 16)
    cell_size = (4, 4)
    block_size_in_cells = (2, 2)
    
    block_size = (block_size_in_cells[1] * cell_size[1], block_size_in_cells[0] * cell_size[0])
    block_stride = (cell_size[1], cell_size[0])
    nbins = 9  # Number of orientation bins
    hog = cv2.HOGDescriptor(win_size, block_size, block_stride, cell_size, nbins)
    h = hog.compute(img)
    h = h.flatten()
    return h.flatten()

In [245]:
def load_dataset(feature_set='hog'):
    features = []
    labels = []
    img_filenames = os.listdir(path_to_dataset)

    for i, fn in enumerate(img_filenames):
        if fn.split('.')[-1] != 'jpg':
            continue

        label = fn.split('.')[0]
        labels.append(label)

        path = os.path.join(path_to_dataset, fn)
        img = cv2.imread(path)
        features.append(extract_features(img, feature_set))
        
        # show an update every 1,000 images
        if i > 0 and i % 1000 == 0:
            print("[INFO] processed {}/{}".format(i, len(img_filenames)))
        
    return features, labels        

In [246]:
def load_dataset():
    (train_images, train_labels), (test_images, test_labels) = mnist.load_data()

    features = []
    labels = []

    for i in range(len(train_images)):
        features.append(extract_features(train_images[i]))
        labels.append(train_labels[i])
    
    for i in range(len(test_images)):
        features.append(extract_features(test_images[i]))
        labels.append(test_labels[i])
    return features, labels      

In [247]:
classifier = KNeighborsClassifier(n_neighbors=5)

In [248]:
def train_model(features, labels):
    train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.2, random_state=random_seed)

    print("############## Training ##############")
    classifier.fit(train_features, train_labels)
    
    # Test the model on images it hasn't seen before
    accuracy = classifier.score(test_features, test_labels)
    
    print('accuracy:', accuracy*100, '%')


In [249]:
features, labels = load_dataset()

In [250]:
train_model(features,labels)

############## Training ##############
accuracy: 98.55000000000001 %


In [251]:
def getNumberFromImage(img):
    img = cleanImage(img)
    result = classifier.predict([extract_features(img)])
    return result[0]


In [253]:
test_image = sk.io.imread("./test8.png")
test_image = cleanImage(test_image)
result = classifier.predict([extract_features(test_image)])
print(result)



[3]
